In [1]:
from pyspark.sql import functions as F
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.types import StringType, StructType, StructField, FloatType
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, udf
from pyspark.ml.feature import Tokenizer, RegexTokenizer
import re
from textblob import TextBlob

In [2]:
#- Création d'une instance du Spark
spark = SparkSession \
    .builder \
    .appName("HespressChallenge") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2") \
    .getOrCreate()

In [3]:
#- Définition de Schema du tableau
schema = StructType([ \
    StructField("author", StringType()), \
    StructField("text", StringType())])

#- Création d'une instance pour le flux des données en temps réel
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "comments_hespress") \
    .load()

values = df.select(from_json(df.value.cast("string"), schema).alias("commentaire"))

In [4]:
df1 = values.select("commentaire.*")
column_author = df1.withColumn('author', col("author"))

In [ ]:
query = column_author.writeStream.outputMode("append").format("console").start()
query.awaitTermination()